# Snowflake Multi-Table Dataset

In this notebook, we will learn how to train a classifier with a more complex multi-table data where a secondary table is itself a parent tables of another table (ie. snowflake schema). It is highly recommended to see the _Sklearn Basics 1_ and _Sklearn Basics 2_ lessons if you are not familiar with pyKhiops' sklearn estimators.

We start by importing pyKhiops sklearn classifier `KhiopsClassifier` and saving the location of the Khiops `Samples` directory into a variable:

In [ ]:
from os import path
import pandas as pd

from khiops import core as kh
from khiops.sklearn import KhiopsClassifier

### Training a Multi-Table Classifier

We'll train a multi-table classifier on a extension of dataset `AccidentsSummary` that we used in the previous notebook Sklearn Basics 2. This dataset `Accidents` contains the additional table `Users` and is organized in the following relational snowflake schema.

```
Accidents
|
| -- 1:n -- Vehicles
              |
              |-- 1:n -- Users
```

Note that the target variable is `Gravity`.

To train the KhiopsClassifier for this setup, we must specify a multi-table dataset. Let's first check the content of the tables:

- The main table `Accidents`
- The secondary table `Vehicles`
- The tertiary table `Users`


In [ ]:
accidents_dataset_dir = path.join(kh.get_samples_dir(), "Accidents")

accidents_file = path.join(
    path.join(kh.get_samples_dir(), "AccidentsSummary"), "Accidents.txt"
)
accidents_df = pd.read_csv(accidents_file, sep="\t", encoding="latin1")
print(f"Accident dataframe (first 10 rows):")
display(accidents_df.head(10))
print()

# We drop the "Gravity" column as it was used to create the target
users_file = path.join(accidents_dataset_dir, "Users.txt")
users_df = pd.read_csv(users_file, sep="\t", encoding="latin1").drop("Gravity", axis=1)
print(f"User dataframe (first 10 rows):")
display(users_df.head(10))
print()

vehicles_file = path.join(accidents_dataset_dir, "Vehicles.txt")
vehicles_df = pd.read_csv(vehicles_file, sep="\t", encoding="latin1")
print(f"Vehicle dataframe (first 10 rows):")
display(vehicles_df.head(10))

#### Create the main feature matrix and the target vector for `Accidents`

In [ ]:
accidents_main_df = accidents_df.drop("Gravity", axis=1)
y_accidents_train = accidents_df["Gravity"]

#### Create the multi-table dataset specification

Note the main table has one key `AccidentId`. Secondary table `Places` has the same key.
Tables `Vehicles` (the other secondary table) and `Users` (the tertiary table) have two keys `AccidentId` and `VehicleId`.

To describe relations between tables, the field `relations` must be added to the dictionary of table specifications. This field
is a list of pairs of tables of the form 
```
(<parent table name>, <child table name>)
``` 

Note that `Places` is here considered as a table as `pykhiops.sklearn` estimators do not support 1:1 relationships at the moment. 

In [ ]:
X_accidents_train = {
    "main_table": "Accidents",
    "tables": {
        "Accidents": (accidents_main_df, "AccidentId"),
        "Vehicles": (vehicles_df, ["AccidentId", "VehicleId"]),
        "Users": (users_df, ["AccidentId", "VehicleId"]),
    },
    "relations": [
        ("Accidents", "Vehicles"),
        ("Vehicles", "Users"),
    ],
}

#### Train a classifier with this dataset

- You may choose the number of features `n_features` to be created by the Khiops AutoML engine
- Set the number of trees to zero (`n_trees=0`)

In [ ]:
pkc_accidents = KhiopsClassifier(n_trees=0, n_features=1000)
pkc_accidents.fit(X_accidents_train, y_accidents_train)

#### Print the accuracy and auc of the model


In [ ]:
accidents_train_performance = (
    pkc_accidents.model_report_.train_evaluation_report.get_snb_performance()
)
print(f"Accidents train accuracy: {accidents_train_performance.accuracy}")
print(f"Accidents train auc     : {accidents_train_performance.auc}")

#### Deploy the classifier to obtain predictions on the training data

Note that usually one deploys the model on new test data. We deploy on the train dataset to keep the tutorial simple*.


In [ ]:
pkc_accidents.predict(X_accidents_train)